- 1) Система совместных рекомендаций

- 2) Система рекомендаций на основе контента

- 3) Система рекомендаций на основе демографических данных

- 4) Система рекомендаций на основе полезности

- 5) Система рекомендаций, основанная на знаниях

- 6) Гибридная система рекомендаций

### Система рекомендаций на основе контента:

![](k1.webp)

По сути, это означает, что на основе контента, который я смотрю на Netflix, я могу запустить алгоритм, чтобы увидеть, какие фильмы наиболее похожи, а затем рекомендовать их другим пользователям. Например, когда вы открываете Amazon и ищете продукт, вы получаете всплывающие под ним похожие продукты, которые представляют собой сходство между товарами, которое они вычислили для всех продуктов, которые есть в Amazon. Это дает нам очень простое, но эффективное представление о том, как продукты ведут себя друг с другом. Хлеб и масло могут быть похожими продуктами в истинном смысле, поскольку они сочетаются друг с другом, но их свойства могут различаться. В случае особенностей киноиндустрии, таких как жанры, обзоры могут рассказать нам о похожих фильмах, и это тип сходства, который мы получаем для фильмов.

### Совместная система рекомендаций:

![](w2.webp)

Когда вы не хотите видеть, каковы особенности продуктов для расчета показателя сходства и проверки взаимодействия продуктов с пользователями, вы называете это совместным подходом. Из взаимодействия продуктов с пользователями мы выясняем, какие продукты похожи, а затем используем стратегию рекомендаций для целевой аудитории. Двух пользователей, которые смотрели один и тот же фильм на Netflix, можно назвать похожими, и когда первый пользователь смотрит другой фильм, второй пользователь получает ту же рекомендацию на основе лайков, которые есть у этих людей.

### Гибридная система рекомендаций:

![](w3.webp)

Является комбинацией нескольких рекомендательныхь систем

### Оценка модели

```AUC Score``` : в бинарном случае (нажатие/не нажатие) оценка AUC имеет хорошую интерпретацию: она выражает вероятность того, что случайно выбранный положительный элемент (элемент, на который нажал пользователь) будет иметь более высокий рейтинг, чем случайно выбранный отрицательный элемент ( элемент, который пользователь не щелкнул). Таким образом, AUC, равная 1,0, означает, что результирующее ранжирование является идеальным: ни один отрицательный элемент не имеет более высокого рейтинга, чем любой положительный элемент.

```Precision@K``` : Precision@K измеряет долю положительных элементов среди K элементов с наивысшим рейтингом. Таким образом, это сосредоточено на качестве рейтинга в верхней части списка: не имеет значения, насколько хороша или плоха остальная часть вашего рейтинга, если первые K пунктов в основном положительные.

Пример: только один элемент из ваших 5 лучших элементов верен, тогда ваша точность @ 5 равна 0,2.

Примечание . Если первых K рекомендуемых товаров больше нет в наличии (скажем, их нет в наличии), вам нужно двигаться дальше вниз по рейтингу. Высокий показатель AUC даст вам уверенность в том, что ваш рейтинг имеет высокое качество.

«Это транснациональный набор данных, который содержит все транзакции, имевшие место в период с 12.01.2010 по 12.09.2011 для британской и зарегистрированной немагазинной онлайн-розничной торговли. Компания в основном продает уникальные подарки на все случаи жизни. Многие клиенты компании являются оптовиками».

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
import pandas as pd
df = pd.read_csv('data.csv', header= 0,
                        encoding= 'unicode_escape')

In [4]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [5]:
df.shape

(541909, 8)

In [6]:
df["StockCode"].nunique()

4070

In [7]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [8]:
df["InvoiceNo"].nunique()

25900

In [9]:
df["CustomerID"].nunique()

4372

In [10]:
df_r=df[["CustomerID","StockCode","Quantity","Description"]]

In [11]:
df_r

,CustomerID,StockCode,Quantity,Description
0,17850.0,85123A,6,WHITE HANGING HEART T-LIGHT HOLDER
1,17850.0,71053,6,WHITE METAL LANTERN
2,17850.0,84406B,8,CREAM CUPID HEARTS COAT HANGER
3,17850.0,84029G,6,KNITTED UNION FLAG HOT WATER BOTTLE
4,17850.0,84029E,6,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...
541904,12680.0,22613,12,PACK OF 20 SPACEBOY NAPKINS
541905,12680.0,22899,6,CHILDREN'S APRON DOLLY GIRL
541906,12680.0,23254,4,CHILDRENS CUTLERY DOLLY GIRL
541907,12680.0,23255,4,CHILDRENS CUTLERY CIRCUS PARADE


In [12]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [13]:
interactions=create_interaction_matrix(df_r,user_col='CustomerID',item_col='StockCode',rating_col='Quantity')

In [15]:
interactions.shape

(4372, 3684)

In [23]:
for i in df_r.columns:
    print(len(df_r[i].unique()))

4373
4070
722
4224


In [24]:
df_r.columns

Index(['CustomerID', 'StockCode', 'Quantity', 'Description'], dtype='object')

In [25]:
interactions

StockCode,23166,16008,17021,20665,20719,20780,20782,20966,21035,21041,...,84689,90104,35271S,44242A,20892,23664,21310,84613C,84206B,21414
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,24.0,36.0,6.0,40.0,12.0,6.0,10.0,6.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [27]:
model=runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4)

In [28]:
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [29]:
item_dict=create_item_dict(df_r,'StockCode','Description')

In [37]:
len(item_dict)

4070

In [38]:
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [39]:
user_dict=create_user_dict(interactions)

In [41]:
len(user_dict)

4372

In [75]:
user_dict[12363.0]

16

In [76]:
interactions

StockCode,23166,16008,17021,20665,20719,20780,20782,20966,21035,21041,...,84689,90104,35271S,44242A,20892,23664,21310,84613C,84206B,21414
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,24.0,36.0,6.0,40.0,12.0,6.0,10.0,6.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
n_users, n_items = interactions.shape
result = model.predict(user_dict[12363.0],np.arange(n_items))

In [64]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Products Already Bought:")
        counter = 1
        for i in known_items:
            try:
                print(str(counter) + '- ' + i)
            except:
                continue
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [65]:
sample_recommendation_user(model, interactions,12363.0, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True)

Products Already Bought:
1- 72 SWEETHEART FAIRY CAKE CASES
2- 60 TEATIME FAIRY CAKE CASES
3- DOORMAT MULTICOLOUR STRIPE
4- BOX OF 6 MINI VINTAGE CRACKERS
5- VINTAGE CHRISTMAS BUNTING
6- STORAGE TIN VINTAGE DOILY 
7- BISCUIT TIN VINTAGE CHRISTMAS
8- 60 CAKE CASES VINTAGE CHRISTMAS
9- 36 DOILIES VINTAGE CHRISTMAS
10- DOORMAT WELCOME SUNRISE
11- PACK OF 60 PINK PAISLEY CAKE CASES
12- PACK OF 60 MUSHROOM CAKE CASES
13- SET OF 36 PAISLEY FLOWER DOILIES
14- SET OF 36 MUSHROOM PAPER DOILIES
15- PACK OF 72 SKULL CAKE CASES
16- PACK OF 72 RETROSPOT CAKE CASES
17- SET OF 72 SKULL PAPER  DOILIES
18- SET OF 72 RETROSPOT PAPER  DOILIES
19- SET/10 BLUE POLKADOT PARTY CANDLES
20- SET/10 IVORY POLKADOT PARTY CANDLES
21- SET/10 PINK POLKADOT PARTY CANDLES
22- SET/10 RED POLKADOT PARTY CANDLES
23- DOORMAT RED RETROSPOT

 Recommended Items:
1- PACK OF 60 SPACEBOY CAKE CASES
2- PACK OF 60 DINOSAUR CAKE CASES
3- SET OF 36 TEATIME PAPER DOILIES
4- 60 CAKE CASES DOLLY GIRL DESIGN
5- SET OF 60 VINTAGE LEAF CA

['22417',
 '21975',
 '84987',
 '22951',
 '23308',
 '21080',
 '23307',
 '22909',
 '85049E',
 '22910']

In [66]:
sample_recommendation_user(model, interactions,12361.0, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True)

Products Already Bought:
1- POSTAGE
2- CIRCUS PARADE LUNCH BOX 
3- DOLLY GIRL LUNCH BOX
4- SPACEBOY LUNCH BOX 
5- PLASTERS IN TIN STRONGMAN
6- LUNCH BAG SPACEBOY DESIGN 
7- ROUND SNACK BOXES SET OF 4 FRUITS 
8- ROUND SNACK BOXES SET OF4 WOODLAND 
9- LUNCH BAG WOODLAND
10- LUNCH BAG RED RETROSPOT

 Recommended Items:
1- PLASTERS IN TIN SPACEBOY
2- PLASTERS IN TIN WOODLAND ANIMALS
3- PLASTERS IN TIN CIRCUS PARADE 
4- LUNCH BAG  BLACK SKULL.
5- RABBIT NIGHT LIGHT
6- LUNCH BAG APPLE DESIGN
7- LUNCH BAG DOLLY GIRL DESIGN
8- RED TOADSTOOL LED NIGHT LIGHT
9- JUMBO BAG SPACEBOY DESIGN
10- JUMBO BAG ALPHABET


['22551',
 '22554',
 '22556',
 '20727',
 '23084',
 '23206',
 '22662',
 '21731',
 '22385',
 '23201']

In [73]:
sample_recommendation_user(model, interactions,12362.0, user_dict, 
                               item_dict,threshold = 30,nrec_items = 10, show = True)

Products Already Bought:
1- POSTAGE
2- LUNCH BAG DOLLY GIRL DESIGN
3- DOLLY GIRL LUNCH BOX
4- SPACEBOY LUNCH BOX 
5- PLASTERS IN TIN WOODLAND ANIMALS
6- MINI PAINT SET VINTAGE 
7- POPCORN HOLDER
8- adjustment

 Recommended Items:
1- CHILDRENS CUTLERY DOLLY GIRL 
2- CHILDRENS CUTLERY SPACEBOY 
3- SPACEBOY CHILDRENS CUP
4- CHILDS BREAKFAST SET SPACEBOY 
5- DOLLY GIRL CHILDRENS CUP
6- ROUND SNACK BOXES SET OF4 WOODLAND 
7- LUNCH BAG SPACEBOY DESIGN 
8- RED RETROSPOT MINI CASES
9- RED RETROSPOT CHILDRENS UMBRELLA
10- LUNCH BAG RED RETROSPOT


['23254',
 '23256',
 '23292',
 '22634',
 '23291',
 '22326',
 '22382',
 '20750',
 '20682',
 '20725']